In [ ]:
import pandas as pd
df = pd.read_excel('/content/ferizy_data.xlsx')
df = df.rename(columns={0: 'text',1: 'label'})
df.head()

,text,label
0,"['buka', 'aplikasi', 'loading', 'memuat', 'dat...",negative
1,"['tolol', 'loading', 'orang', 'bayar']",negative
2,"['aplikasi', 'buruk', 'menyusahkan']",negative
3,['bagus'],positive
4,"['minimal', 'database', 'pakai', 'jaringan', '...",negative


In [ ]:
df.shape

(1500, 2)

In [ ]:
df.label.value_counts()

negative    915
positive    585
Name: label, dtype: int64

In [ ]:
import re

def case_folding(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return string

In [ ]:
def case_folding(data):
  temp_ulasan = []

  for mm in data:
    # removal of links[https://example.com]
    # mm = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", mm)
    mm = re.sub(r"http\S+", "", mm)
    mm = re.sub(r"'", "", mm)
    # removal of new line
    mm = re.sub('\n', '', mm)

    # removal of punctuations and numbers
    mm = re.sub("[^a-zA-Z^']", " ", mm)
    mm = re.sub(" {2,}", " ", mm)

      # remove whitespace with a single space
    mm = re.sub(r'\s+', ' ', mm)

    # convert text to Lowercase
    mm = mm.lower();
    temp_ulasan.append(mm)
  return temp_ulasan

In [ ]:
df['text_clean'] = case_folding(df['text'])
df.head()

,text,label,text_clean
0,"['buka', 'aplikasi', 'loading', 'memuat', 'dat...",negative,buka aplikasi loading memuat data selesai per...
1,"['tolol', 'loading', 'orang', 'bayar']",negative,tolol loading orang bayar
2,"['aplikasi', 'buruk', 'menyusahkan']",negative,aplikasi buruk menyusahkan
3,['bagus'],positive,bagus
4,"['minimal', 'database', 'pakai', 'jaringan', '...",negative,minimal database pakai jaringan tersendiri ca...


In [ ]:
neg = df.loc[df['label'] == 'negative'].text_clean.tolist()
pos = df.loc[df['label'] == 'positive'].text_clean.tolist()

neg_label = df.loc[df['label'] == 'negative'].label.tolist()
pos_label = df.loc[df['label'] == 'positive'].label.tolist()


In [ ]:
total_data = pos + neg
labels = pos_label + neg_label

print("Pos: %s, Neg: %s" % (len(pos), len(neg)))
print("Total data: %s" % len(total_data))

Pos: 585, Neg: 915
Total data: 1500


In [ ]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict

max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(total_data)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("tokenizer.pickle has created!")

X = tokenizer.texts_to_sequences(total_data)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X)

X = pad_sequences(X)
with open('x_pad_sequences.pickle', 'wb') as handle:
    pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("x_pad_sequences.pickle has created!")

tokenizer.pickle has created!
x_pad_sequences.pickle has created!


In [ ]:
Y = pd.get_dummies(labels)
Y = Y.values

with open('y_labels.pickle', 'wb') as handle:
    pickle.dump(Y, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("y_labels.pickle has created!")

y_labels.pickle has created!


In [ ]:
from sklearn.model_selection import train_test_split

file = open("x_pad_sequences.pickle",'rb')
X = pickle.load(file)
file.close()

file = open("y_labels.pickle",'rb')
Y = pickle.load(file)
file.close()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K

In [ ]:
embed_dim = 100
units = 64

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(LSTM(units, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

adam = optimizers.Adam(learning_rate = 0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test), verbose=1, callbacks=[es])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 57, 100)           10000000  
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 10042370 (38.31 MB)
Trainable params: 10042370 (38.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
135/135 [==============================] - 36s 249ms/step - loss: 0.5103 - accuracy: 0.7785 - val_loss: 0.4150 - val_accuracy: 0.8200
Epoch 2/10
135/135 [==============================] - 34s 250ms/step - loss: 0.2904 - accuracy: 0.8822 - val_loss: 0.3262 - val_

In [ ]:
from sklearn import metrics

predictions = model.predict(X_test)
y_pred = predictions
matrix_test = metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Testing selesai")
print(matrix_test)

5/5 [==============================] - 1s 14ms/step
Testing selesai
              precision    recall  f1-score   support

           0       0.87      0.91      0.89        87
           1       0.86      0.81      0.84        63

    accuracy                           0.87       150
   macro avg       0.87      0.86      0.86       150
weighted avg       0.87      0.87      0.87       150

